In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # original code - kept for reference.
        # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input: 28; Output: 28, RF: 3
        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input: 28; Output: 28, RF: 5
        # self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        # # self.fc1 = nn.Linear(in_features = 1792 * 14, out_features = 64)
        # self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input: 14; Output: 14, RF: 10
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input: 14; Output: 14, RF: 14
        # self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        # self.conv5 = nn.Conv2d(256, 512, 3) #input: 7; Output: 5, RF: 24
        # self.conv6 = nn.Conv2d(512, 1024, 3) #input: 5; Output: 3, RF: 32
        # self.conv7 = nn.Conv2d(1024, 10, 3) #input: 3; Output: 1, RF: 40

        # reducing channels to 8, exploring with BN
        # self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input: 28; Output: 28, RF: 3
        # self.bn1 = nn.BatchNorm2d (8)
        # self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #input: 28; Output: 28, RF: 5
        # self.bn2 = nn.BatchNorm2d (16)
        # self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        # self.antman1 = nn.Conv2d(16, 16, 1)
        # self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #input: 14; Output: 14, RF: 10
        # self.bn3 = nn.BatchNorm2d (32)
        # self.conv4 = nn.Conv2d(32, 64, 3, padding=1) #input: 14; Output: 14, RF: 14
        # self.bn4 = nn.BatchNorm2d (64)
        # self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        # self.antman2 = nn.Conv2d(64, 64, 1)
        # self.conv5 = nn.Conv2d(64, 128, 3) #input: 7; Output: 5, RF: 24
        # self.bn5 = nn.BatchNorm2d (128)
        # self.conv6 = nn.Conv2d(128, 256, 3) #input: 5; Output: 3, RF: 32
        # self.bn6 = nn.BatchNorm2d (256)
        # self.conv7 = nn.Conv2d(256, 10, 3) #input: 3; Output: 1, RF: 40
        # #self.dropout = nn.Dropout(0.1)

        # adding antman after MP to reduce output
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input: 28; Output: 28, RF: 3
        self.bn1 = nn.BatchNorm2d (8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #input: 28; Output: 28, RF: 5
        self.bn2 = nn.BatchNorm2d (16)
        self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        self.antman1 = nn.Conv2d(16, 8, 1)

        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input: 14; Output: 14, RF: 10
        self.bn3 = nn.BatchNorm2d (16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input: 14; Output: 14, RF: 14
        self.bn4 = nn.BatchNorm2d (32)
        self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        self.antman2 = nn.Conv2d(32, 8, 1)

        self.conv5 = nn.Conv2d(8, 16, 3) #input: 7; Output: 5, RF: 24
        self.bn5 = nn.BatchNorm2d (16)
        self.conv6 = nn.Conv2d(16, 32, 3) #input: 5; Output: 3, RF: 32
        self.bn6 = nn.BatchNorm2d (32)
        self.conv7 = nn.Conv2d(32, 10, 3) #input: 3; Output: 1, RF: 40
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # commenting out original, and breaking it up to be able to mix and match and test.
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn2(x)

        x = self.pool1(x)
        x = self.antman1(x)

        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # commenting out original, and breaking it up to be able to mix and match and test.
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn3(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn4(x)

        x = self.pool2(x)
        x = self.antman2(x)

        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # commenting out original, and breaking it up to be able to mix and match and test.
        x = self.conv5(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn5(x)

        x = self.conv6(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.bn6(x)

        # x = F.relu(self.conv7(x))
        x = self.conv7(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [12]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
           Dropout-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 16, 28, 28]           1,168
           Dropout-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             136
            Conv2d-9           [-1, 16, 14, 14]           1,168
          Dropout-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 32, 14, 14]           4,640
          Dropout-13           [-1, 32, 14, 14]               0
      BatchNorm2d-14           [-1, 32,

<ipython-input-11-d14b2b23b027>:105: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-11-d14b2b23b027>:105: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06969023495912552 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.54it/s]



Test set: Average loss: 0.0610, Accuracy: 9828/10000 (98%)



loss=0.016963116824626923 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.65it/s]



Test set: Average loss: 0.0462, Accuracy: 9866/10000 (99%)



loss=0.008798043243587017 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.96it/s]


...


Test set: Average loss: 0.0192, Accuracy: 9942/10000 (99%)



loss=0.0030432420317083597 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.69it/s]



Test set: Average loss: 0.0165, Accuracy: 9948/10000 (99%)



loss=0.06604822725057602 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.54it/s]



Test set: Average loss: 0.0200, Accuracy: 9934/10000 (99%)

